In [60]:
#Enviroment set-up
import csv
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, LogisticRegression
from scipy.signal import savgol_filter


STOCK_TICKER = "plug"

# Clean Data

- Remove comma from volume that messes with the delimiter of the CSV file
- Transfer data from CSV file to NP array

In [61]:
#Clean data, specifically 
with open(r'Data Collection\plug.csv', "r") as infile, open("plug_output.csv", "w") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        writer.writerow(item.replace(",", "") for item in row)

        
#my_data = np.genfromtxt(r'Data Collection\plug.csv')
data = []
with open(r'plug_output.csv', "r") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        temp = []
        for item in row:
            temp.append(item)

        if(len(temp) == 0):
            continue
        data.append(temp)

        
dataNP = np.asarray(data) #(3964, 6)

# Feature Engineering & Manipulation
- Seperate features\labels
- Feature Matrix (shape - (4, 3963))
    - price_spread
    - volume
    - high_volume --> total money movement assuming that all volume as sold at the high of the day
    - low_volume --> total money movement assuming that all volume as sold at the low of the day
- Label Matrix (shape - (2, 3963))

# TODO
- Feature engineering research

In [62]:
date = dataNP[:, 0][1:]
openPrice = dataNP[:, 1][1:].astype(np.float)
high = dataNP[:, 2][1:].astype(np.float)
low = dataNP[:, 3][1:].astype(np.float)
close = dataNP[:, 4][1:].astype(np.float)

#creating features (3963,)
price_spread = high - low
volume = dataNP[:, 5][1:].astype(np.float)
high_volume = high * volume
low_volume = low * volume                     

#feature matrix
feature_matrix = np.vstack((price_spread, volume, high_volume, low_volume)).T
#label matrix
label_matrix = openPrice.T




# Regression Model

In [68]:
class Regression(object):

    def __init__(self):
        pass

    def rmse(self, pred, label):
        return np.sqrt((np.sum((pred - label)**2)/pred.shape[0]))

    def construct_polynomial_feats(self, x, degree):
        feat = np.zeros((x.shape[0], degree + 1))
        for j in range(x.shape[0]):
            for i in range(degree + 1):
                feat[j][i] = x[j]**i
        return feat 
            
    def predict(self, xtest, weight):
        return np.dot(xtest, weight)

    def linear_fit_closed(self, xtrain, ytrain):
        return np.dot(np.linalg.pinv(xtrain), ytrain)


In [69]:
def apply_regression(X_train, y_train, X_test):
    ridge = Ridge()
    weight = ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    
    return y_pred

In [71]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, label_matrix, test_size=.3, random_state=42)
y_pred = apply_regression(X_train, y_train, X_test)

# calculate RMSE 
rmse_score = np.sqrt(mean_squared_error(y_pred, y_test))
print("rmse score", rmse_score)

rmse score 3.9268978632990788


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.6362e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
